In [4]:
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, FloatSlider, Output
from shapely.geometry import Polygon

space = np.linspace(0, 1, 1000)

First, let's consider a single transition in standard PLA where we are selecting $p$ against $1-p$.

<img src="1.jpeg" width="400"/>

In [ ]:
def plot_with_region(lower_bound, upper_bound):
    fig, ax = plt.subplots()
    ax.set_xlim([0.0, 1.0])
    ax.set_xlabel("prob to reach $s_1$ from $s_0$")
    ax.set_ylim([0.0, 1.0])
    ax.set_ylabel("prob to reach $s_2$ from $s_0$")
    ax.plot(space, 1 - space)
    ax.plot(lower_bound, 1 - lower_bound, "bo")
    ax.plot(upper_bound, 1 - upper_bound, "bo")
interact(plot_with_region, lower_bound=(0.0, 1.0, 0.01), upper_bound=(0.0, 1.0, 0.01))
    

Now let's introduce some parameter dependencies. Consider this situation:

<img src="2.jpeg" width="400"/>

First, we will see how the probability to reach selected states relate when using standard parameter lifting and when using big-step lifting.

In [12]:

plt.ion()
def plot_with_region(lower_bound, upper_bound):
    figs, axs = plt.subplots(1,2, figsize=(14, 7))
    for ax in axs:
        ax.set_xlim([0.0, 1.0])
        ax.set_xlabel("prob to reach $s_2$ from $s_0$ ($=a$)")
        ax.set_ylim([0.0, 1.0])
        ax.set_ylabel("prob to reach $s_3$ from $s_0$ ($=b$)")
        ax.plot(space * space, space * (1 - space))
        ax.plot(lower_bound * lower_bound, lower_bound * (1 - lower_bound), "bo")
        ax.plot(upper_bound * upper_bound, upper_bound * (1 - upper_bound), "bo")
    
    # standard PLA
    axs[0].set_title("Standard PLA")
    x1 = lower_bound * lower_bound
    x2 = lower_bound * upper_bound
    x3 = upper_bound * upper_bound
    x4 = upper_bound * lower_bound
    y1 = lower_bound * (1 - lower_bound)
    y2 = lower_bound * (1 - upper_bound)
    y3 = upper_bound * (1 - upper_bound)
    y4 = upper_bound * (1 - lower_bound)
    axs[0].fill([x1, x2, x3, x4, x1], [y1, y2, y3, y4, y1], "red", alpha=0.5)

    axs[1].set_title("Big-Step Lifting")
    
    # box just with a and b
    min_a = lower_bound * lower_bound
    max_a = upper_bound * upper_bound
    min_b = min(lower_bound * (1 - lower_bound), upper_bound * (1 - upper_bound))
    max_b = 0.25 if lower_bound < 0.5 and upper_bound > 0.5 else max(lower_bound * (1 - lower_bound), upper_bound * (1 - upper_bound))
    min_a = min(min_a, upper_bound - min_b)
    
    pol1 = Polygon([(min_a, min_b), (min_a, max_b), (max_a, max_b), (max_a, min_b)])
    pol2 = Polygon([(upper_bound, 0), (0, upper_bound), (0, lower_bound), (lower_bound, 0)])
    
    def plot_polygon(pol):
        x = []
        y = []
        for v in pol.exterior.coords:
            x.append(v[0])
            y.append(v[1])
        x.append(pol.exterior.coords[0][0])
        y.append(pol.exterior.coords[0][1])
        axs[1].fill(x, y, "green", alpha=0.5)
    plot_polygon(pol1.intersection(pol2))
    plt.show()
    
interact(plot_with_region, lower_bound=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2), upper_bound=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.7))
plt.close()

interactive(children=(FloatSlider(value=0.2, description='lower_bound', max=1.0, step=0.01), FloatSlider(value…

In the above plot, this is how I got the polygon for big-step lifting: First, collapse the transitions and give new names to the probabilities:

<img src="3.jpeg" width="400"/>

Now, we have these constraints:

(1) $\min(p^2) \leq a \leq \max(p^2) \implies l_b^2 < a < u_b^2$

(2) $\min(p \cdot (1-p)) < b < \max(p \cdot (1-p)) \implies $ case distinction expression of bounds for $b$

(3) $\min(1-p) \leq 1 - a - b \leq \max(1-p) \implies l_b \leq a + b \leq u_b$

(1) and (2) form a rectangle and (3) forms a polygon with four vertices, intersecting these leads to the final convex polygon.

Let's now consider the case where a state has two successors that are $p$-transitions. First, we do a constant transformation like this:

<img src="4.jpeg" width="400"/>

There is only one $p$-transition pair after $p$ and one after $1-p$ because of time-travelling, so this is the most complex situation we have to handle. Now, let's do the renaming thing again, and let's consider a simplified scenario with no constants, as we have seen that we can weed them out:

<img src="5.jpeg" width="700"/>

Now, we have these constraints:

(1) $\min(p^2) \leq a \leq \max(p^2) \implies l_b^2 < a < u_b^2$

(2) $\min(2 \cdot (p \cdot (1-p))) < b < \max(2 \cdot (p \cdot (1-p))) \implies $ case distinction expression of bounds for $b$

(3) $\min((1-p)^2) \leq 1 - a - b \leq \max((1-p)^2) \implies (1 - u_b)^2 \leq 1 - a - b \leq (1 - l_b)^2 \implies 1 - (1 - l_b)^2 \leq a + b \leq 1 - (1 - u_b)^2$

(1) and (2) form a rectangle and (3) forms a polygon with four vertices, intersecting these leads to the final convex polygon.


In [18]:
def plot_with_region(lower_bound, upper_bound):
    figs, axs = plt.subplots(1,2, figsize=(14, 7))
    for ax in axs:
        ax.set_xlim([0.0, 1.0])
        ax.set_xlabel("prob to reach $s_1$ from $s_0$ ($=a$)")
        ax.set_ylim([0.0, 1.0])
        ax.set_ylabel("prob to reach $s_3$ from $s_0$ ($=b$)")
        ax.plot(space * space, space * (1 - space))
        ax.plot(lower_bound * lower_bound, lower_bound * (1 - lower_bound), "bo")
        ax.plot(upper_bound * upper_bound, upper_bound * (1 - upper_bound), "bo")
    
    def plot_polygon(pol, ax, color="green"):
        x = []
        y = []
        for v in pol.exterior.coords:
            x.append(v[0])
            y.append(v[1])
        x.append(pol.exterior.coords[0][0])
        y.append(pol.exterior.coords[0][1])
        ax.fill(x, y, color, alpha=0.5)
    # standard PLA
    axs[0].set_title("Standard PLA")
    x = [
        lower_bound * lower_bound,
        lower_bound * lower_bound,
        lower_bound * upper_bound,
        lower_bound * upper_bound,
        upper_bound * upper_bound,
        upper_bound * upper_bound,
        upper_bound * lower_bound,
        upper_bound * lower_bound
    ]
    y = [
        (1 - lower_bound) * lower_bound,
        (1 - lower_bound) * upper_bound,
        (1 - lower_bound) * upper_bound,
        (1 - lower_bound) * lower_bound,
        (1 - upper_bound) * lower_bound,
        (1 - upper_bound) * upper_bound,
        (1 - upper_bound) * upper_bound,
        (1 - upper_bound) * lower_bound,
    ]
    pol = Polygon([[x[i], y[i]] for i in range(len(x))]).convex_hull
    plot_polygon(pol, axs[0], color="red")

    axs[1].set_title("Big-Step Lifting")
    
    # box just with a and b
    min_a = lower_bound * lower_bound
    max_a = upper_bound * upper_bound
    min_b = 2 * min(lower_bound * (1 - lower_bound), upper_bound * (1 - upper_bound))
    max_b = 0.5 if lower_bound < 0.5 and upper_bound > 0.5 else 2 * max(lower_bound * (1 - lower_bound), upper_bound * (1 - upper_bound))
    min_a = min(min_a, upper_bound - min_b)
    
    pol1 = Polygon([(min_a, 0.5 * min_b), (min_a, 0.5 * max_b), (max_a, 0.5 * max_b), (max_a, 0.5 * min_b)])#

    u_a = 1 - (1 - upper_bound) * (1 - upper_bound)
    l_a = 1 - (1 - lower_bound) * (1 - lower_bound)
    pol2 = Polygon([(u_a, 0), (0, 0.5 * u_a), (0, 0.5 * l_a), (l_a, 0)])
    
    plot_polygon(pol1.intersection(pol2), axs[1])
    
i = interact(plot_with_region, lower_bound=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2), upper_bound=FloatSlider(min=0.0, max=1.0, step=0.01, value=0.7))


interactive(children=(FloatSlider(value=0.2, description='lower_bound', max=1.0, step=0.01), FloatSlider(value…